# Gemma 3 4B QNN model conversion with Olive 
### Task: Text + Vision Generation 📝

In this notebook, you'll:
- Download the required datasets
- Quantize language model
- Convert Vision to QNN format


### Platform requirements
This notebook is intended to run on a machine with:
  * **Operating System**: Linux Ubuntu 22.04 (automated setup script is Linux-only)
  * **Python**: 3.10
  * NVIDIA driver version equivalent to 525.60.13
  * NVIDIA A100 GPU
  * **Storage**: ~13GB for COCO train2017 dataset (downloaded automatically)

🐍 Python Virtual environments
Creates Olive and QNN python virtual environments

In [ ]:
!git clone https://github.com/CodeLinaro/Olive.git -b dev/qti-tbhardwa/gemma3

In [ ]:
import os
import venv
from pathlib import Path
import subprocess
import json
import shutil
import urllib.request
import onnx
from onnx import helper, TensorProto
import glob

current_dir = os.getcwd()
MODEL="google/gemma-3-4b-it"
OLIVE_PYTHON_PATH = './olive_venv'
OLIVE_PYTHON_BIN = './olive_venv/bin/python'
olive_pip_path = Path(OLIVE_PYTHON_PATH) / "bin" / "pip"
OLIVE_REPO_PATH = Path("./Olive")
OLIVE_REQ = "./requirements.txt"
QNN_REQ = "./qnn_req.txt"

QNN_PYTHON_PATH = './qnn_venv'
QNN_PYTHON_BIN_PATH = './qnn_venv/bin'
qnn_pip_path = Path(QNN_PYTHON_PATH) / "bin" / "pip"
QNN_PYTHON_BIN_FULL_PATH = f"{current_dir}/{QNN_PYTHON_BIN_PATH}"

### Prepare Olive Python Environment

In [ ]:
if not os.path.exists(OLIVE_PYTHON_PATH):
    print("Creating Olive Venv")
    builder = venv.EnvBuilder(with_pip=True)
    builder.create(Path(OLIVE_PYTHON_PATH))
my_env = os.environ.copy()
my_env["BUILD_CUDA_EXT"] = "0"
GPTQ="git+https://github.com/ModelCloud/GPTQModel.git@558449bed3ef2653c36041650d30da6bbbca440d"
subprocess.check_call([str(olive_pip_path), "install", "-U", "-r" , OLIVE_REQ], env=my_env)
subprocess.check_call([str(olive_pip_path), "install", "--no-build-isolation", GPTQ], env=my_env)
subprocess.check_call([str(olive_pip_path), "install", "-e", OLIVE_REPO_PATH])
subprocess.check_call([str(olive_pip_path), "install", "onnxscript==0.5.6"])

### Prepare QNN Python Environment

In [ ]:
if not os.path.exists(QNN_PYTHON_PATH):
    print("Creating QNN Venv")
    builder = venv.EnvBuilder(with_pip=True)
    builder.create(Path(QNN_PYTHON_PATH))
subprocess.check_call([str(qnn_pip_path), "install", "--no-build-isolation", "-r" , QNN_REQ], env=my_env)
subprocess.check_call([str(qnn_pip_path), "install", "-e", OLIVE_REPO_PATH])
subprocess.check_call([str(qnn_pip_path), "install", "-U", "--pre", "--extra-index-url",
                       "https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ORT-Nightly/pypi/simple",
                       "onnxruntime-qnn==1.23.2", "--no-deps"])

### 🤗 Login to Hugging Face
To access models, you'll need to log-in to Hugging Face with a [user access token](https://huggingface.co/docs/hub/security-tokens). The following command will run you through the steps to login:

In [ ]:
!huggingface-cli login --token <>

### Apply few patches to Onnxruntime and GPTQModel

This is needed for running the Olive recipies for this model

In [ ]:
!patch ./olive_venv/lib/python3.10/site-packages/gptqmodel/utils/model.py < gptqmodel_int8.patch
!patch ./olive_venv/lib/python3.10/site-packages/onnxruntime_genai/models/builder.py < oga_patch1.patch
!patch ./olive_venv/lib/python3.10/site-packages/onnxruntime_genai/models/quantized_model.py < oga_patch2.patch

In [ ]:
base_url = "https://raw.githubusercontent.com/CodeLinaro/onnxruntime/326d9d30129bbad698e0306d24dcea0ec5a19e60"
urls = [
    base_url + "/onnxruntime/python/tools/quantization/execution_providers/qnn/quant_config.py",
    base_url + "/onnxruntime/python/tools/quantization/quant_utils.py"
]

destinations = [
    OLIVE_PYTHON_PATH+"/lib/python3.10/site-packages/onnxruntime/quantization/execution_providers/qnn/quant_config.py",
    OLIVE_PYTHON_PATH+"/lib/python3.10/site-packages/onnxruntime/quantization/quant_utils.py"
]

for url, dest in zip(urls, destinations):
    urllib.request.urlretrieve(url, dest)
    print(f"Downloaded and replaced: {dest}")

## Run Olive Recipes

**GPU utilization observed during the run**

		a. Text GPTQModel quantization:        12gb
		b. Text Onnx static quantization:      41gb
		c. Vision Onnx static quantization:    68gb
        d. Embedding Onnx static quantization: 3gb

In [ ]:
# Clean Context binary directories if they exist
def clean_directory(path):
    if os.path.exists(path):
        for file in glob.glob(os.path.join(path, '*')):
            if os.path.isfile(file):
                os.remove(file)
dirs_to_clean = [
    './models/gemma3_qnn/model/',
    './models/gemma-3-4b-it-vision/model/',
    './models/gemma-3-4b-it-embed/model/'
]

for dir_path in dirs_to_clean:
    clean_directory(dir_path)

### 1️⃣ LLM model generation

In [ ]:
config_path = Path(f"./gemma3-4b-text-qnn-config.json")
with open(config_path, "r") as file:
    data = json.load(file)

data["systems"]["qnn_system"]["python_environment_path"] = QNN_PYTHON_BIN_FULL_PATH
data["input_model"]["model_path"] = MODEL

with open(config_path, "w") as file:
    json.dump(data, file, indent=4)

In [ ]:
!./olive_venv/bin/olive run --config ./gemma3-4b-text-qnn-config.json
# Do weight sharing in context and iterator model
!python share_data.py
!unlink models/gemma3_qnn/model/iterator.onnx.data

### 2️⃣ Vision model Quantization

In [ ]:
config_path = Path(f"./gemma3-4b-vision-qnn-config.json")
with open(config_path, "r") as file:
    data = json.load(file)
data["systems"]["qnn_system"]["python_environment_path"] = QNN_PYTHON_BIN_FULL_PATH

with open(config_path, "w") as file:
    json.dump(data, file, indent=4)

In [ ]:
!./olive_venv/bin/olive run --config ./gemma3-4b-vision-export.json
!python vision_allowzero.py
!./olive_venv/bin/olive run --config ./gemma3-4b-vision-qnn-config.json
!rm -rf models/gemma-3-4b-it-vis-onnx/

### 3️⃣ Embedding Model

In [ ]:
!./olive_venv/bin/olive run --config ./gemma3-4b-embedding-qnn-config.json

### Prepare final ORT GenAI folder for on-device inference 

In [ ]:
# copy quant emb weight in to vision emb model
!python copy_embed_quant_param.py

!rm ./models/gemma3_qnn/model/embeddings.onnx
!cp ./models/gemma-3-4b-it-vision/model/model_updated_ctx.onnx ./models/gemma3_qnn/model/model_ctx_vision.onnx 
!cp ./models/gemma-3-4b-it-vision/model/model_updated_ctx_qnn.bin ./models/gemma3_qnn/model/model_updated_ctx_qnn.bin 
!cp ./genai/*.* ./models/gemma3_qnn/model/
!ls -al ./models/gemma3_qnn/model/

print("ORT GenAI inference setup: ./models/gemma3_qnn")